<a href="https://colab.research.google.com/github/adc257/info4940-sitcom/blob/main/may7%3D14_5nContext_LLAMA_testing_100tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import random
import json
import os

In [4]:
!git clone https://github.com/adc257/info4940-sitcom.git


from google.colab import drive
drive.mount('/content/drive')

fatal: destination path 'info4940-sitcom' already exists and is not an empty directory.
Mounted at /content/drive


# Defining some helper functions

In [5]:
def list_files(start_path):
    file_paths = []
    for root, dirs, files in os.walk(start_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    file_paths.sort()
    return file_paths

def fPathList_TO_DialogueWithContext(filePathList, nSentenceContext):
  humorous_inputs = []
  non_humorous_inputs = []

  # formatted_inputs = []
  # labels = []

  for episodePath in filePathList:

    with open(episodePath, 'r') as file:
      data = json.load(file)
      data = [info for (_, info) in data.items()]


      i = nSentenceContext
      while i < len(data):
        context = [data[j] for j in range(i-nSentenceContext, i)]
        target = data[i]

        if 'isHumor' in target:
          humorous_inputs.append(transformInput(context, target))
        else:
          non_humorous_inputs.append(transformInput(context, target))

        i += 1

  return humorous_inputs, non_humorous_inputs

def sampleEven(humorousData, nonHumorousData, sample_size_per_class, rand_seed):
  sample_size_per_class = min(sample_size_per_class, len(humorousData), len(nonHumorousData))

  sample_inputs = []
  sample_labels = []

  data = humorousData
  test_size = (len(data) - sample_size_per_class) / len(data)
  sample, _ = train_test_split(data, test_size=test_size, random_state=rand_seed)
  sample_inputs += sample
  sample_labels += [1]*len(sample)


  data = nonHumorousData
  test_size = (len(data) - sample_size_per_class) / len(data)
  sample, _ = train_test_split(data, test_size=test_size, random_state=rand_seed)
  sample_inputs += sample
  sample_labels += [0]*len(sample)

  return sample_inputs, sample_labels



In [6]:
def testLLAMA(inputs, labels, printResultsEveryN=50):
  resultsData = {
      "truePos": [],
      "trueNeg": [],
      "falsePos": [],
      "falseNeg": [],
      "other": [],
  }

  results = {
      "truePos": 0,
      "trueNeg": 0,
      "falsePos": 0,
      "falseNeg": 0,
      "other": 0,
  }

  correct = 0
  total = 0
  pred = []



  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  for i in range(len(inputs)):
    input = inputs[i]
    label = labels[i]

    input_ids = tokenizer.apply_chat_template(
      input,
      add_generation_prompt=True,
      return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
      input_ids,
      max_new_tokens=50,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
      num_return_sequences=1,
    )

    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)


    if str(response) == str(label):
      # true
      if str(label) == "1":
        # pos
        resultsData["truePos"].append(input)
        results["truePos"] += 1
      elif str(label) == "0":
        # neg
        resultsData["trueNeg"].append(input)
        results["trueNeg"] += 1
      else:
        # other
        resultsData["other"].append(input)
        results["other"] += 1
      correct += 1
    else:
      # false
      if str(label) == "1":
        # pos, pred neg
        resultsData["falseNeg"].append(input)
        results["falseNeg"] += 1
      elif str(label) == "0":
        # neg, pred pos
        resultsData["falsePos"].append(input)
        results["falsePos"] += 1
      else:
        # other
        resultsData["other"].append(input)
        results["other"] += 1
    pred.append(response)
    total += 1

    if i % printResultsEveryN == 0:
      print(f"Test: {i+1} of  {len(inputs)}")
      print(f"Success rate: {correct / total:.2%}\n")
      print("current results:", results)


  print(f"Final success rate: {correct / total:.2%}")
  print("final results:", results)

  # true = [str(i) for i in labels]
  report = None # classification_report(y_pred=pred, y_true=true, target_names=["Non-Humorous","Humorous"])

  return results, resultsData, pred, report

## Defining input formatter

In [7]:
def transformInput(context, target):
  messages = []

  separator = "\n"
  quotation_token = '"'

  # formatted_input = ""

  if (len(context) > 0):
    # if context...
    systemPrmpt = {
        'role': 'system',
        'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."
    }
  else:
    # if no context...
    systemPrmpt = {
        'role': 'system',
        'content': "You are humor classification model that takes a given line and determines whether it was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."
    }

  # append systemPrmpt
  messages.append(systemPrmpt)

  systemPrmpt2 = {
      'role': 'system',
      'content': ""
  }
  userPrmpt = {
      'role': 'user',
      'content': ""
  }
  userPrmpt2 = {
      'role': 'user',
      'content': ""
  }
  asstPrmpt = {
      'role': 'assistant',
      'content': ""
  }

  # append userPrmpt
  # messages.append(userPrmpt)

  if len(context) > 0:
    # asstPrmpt['content'] += "Can you provide some context to help me with this decision?"
    # append asstPrmpt
    # messages.append(asstPrmpt)

    userPrmpt['content'] += "Use the following as context:" + separator*2

    # if entry['Scene'] != current_scene:
    #   current_scene = entry["Scene"]
    # userPrmpt['content'] += "The scene changes to: "
    # else:

    entry = context[0]
    current_scene = entry["Scene"]
    userPrmpt['content'] += "The Scene is: "
    userPrmpt['content'] += current_scene + separator

    for entry in context:
      if current_scene != entry['Scene']:
        current_scene = entry["Scene"]
        userPrmpt['content'] += "The scene changes to: " + current_scene + separator

      recipients = entry['Recipients']
      recipients_str = ""
      if len(recipients) == 0:
        recipients_str += "to themselves"
      elif len(recipients) == 1:
        recipients_str += recipients[0]
      else:
        recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

      userPrmpt['content'] += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
      userPrmpt['content'] += separator

    # append userPrmpt2

  entry = target
  if len(context) > 0:
    # with context
    userPrmpt['content'] += separator * 2 + "Based on the context provided, is the following line from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:"

    current_scene = context[-1]['Scene']
    userPrmpt['content'] += separator*2


    if entry['Scene'] != current_scene:
      current_scene = entry["Scene"]
      userPrmpt['content'] += "The scene changes to: "
    else:
      userPrmpt['content'] += "The Scene is still: "

    userPrmpt['content'] += current_scene + separator

    recipients_str = ""
    recipients = entry['Recipients']
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    userPrmpt['content'] += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."

  else:
    # no context
    userPrmpt['content'] += "Is the following line humorous or not humorous? Only respond '1' for humorous or '0' for not humorous: "
    userPrmpt['content'] += f"{quotation_token}{entry['Dialogue']}{quotation_token}."

  messages.append(userPrmpt)
  return messages

### Sample Raw and Cleaned Formats:

In [8]:
filePath = "/content/info4940-sitcom/cleaned-data/S1/The Big Bang_S0101.json"
nSentenceContext = 5

with open(filePath, 'r') as file:
  data = json.load(file)
  data = [info for (_, info) in data.items()]

print("NCONTEXT: ", nSentenceContext)

print(f"Example raw entry:")
display(data[nSentenceContext])

print(f"\n\nExample formatted entry ({nSentenceContext}context):\n")
input = transformInput(data[:nSentenceContext], data[nSentenceContext])

for i in input:
  print(f"Role: {i['role']} | Content:\n{i['content']}\n")

print(f'Actual: {("isHumor" in data[3])}')

NCONTEXT:  5
Example raw entry:


{'EpisodeID': 'The Big Bang_S0101',
 'Scene': 'A corridor at a sperm bank.',
 'Recipients': ['Sheldon', 'Receptionist'],
 'Speaker': 'Leonard',
 'Dialogue': '14 down is... Move your finger...',
 'Dialogue Start Time': '00:00:35:120000',
 'Dialogue End Time': '00:00:37:190000',
 'isHumor': True,
 'humorDuration': '0:00:01'}



Example formatted entry (5context):

Role: system | Content:
You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0.

Role: user | Content:
Use the following as context:

The Scene is: A corridor at a sperm bank.
Sheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it's unobserved, it will. However, if it's observed after it's left the plane but before it hits its target, it won't have gone through both slits." to Leonard.
Leonard says "Agreed. What's your point?" to Sheldon.
Sheldon says "There's no point, I just think it's a good idea for a T-shirt." to Leonard.
Leonard says "Excuse me." to Sheldon and Receptionist.
Leonard says "One across is Aegean. Eight down is Nabokov. 26 across is MCM." to She

# Creating sample data

In [9]:
filePaths = []
for i in range(5):
  fPath = '/content/info4940-sitcom/cleaned-data/S' + str(i+1)
  filePaths += list_files(fPath)

# getting random number for random state for consistency?
rand_seed1 = 64 # random.randint(1,100)
rand_seed2 = 94 # random.randint(1,100)
rand_seed3 = 69 # random.randint(1,100)

# print([rand_seed1,rand_seed2,rand_seed3])

# sampling 10 episodes
nSamples = 10
test_size = (len(filePaths) - nSamples) / len(filePaths)
samplePaths, _ = train_test_split(filePaths, test_size=test_size, random_state=rand_seed1)

# cleaning/transforming inputs
nContext = 5
nTests = 100

humorousData, nonHumorousData = fPathList_TO_DialogueWithContext(samplePaths, nContext)
sample_inputs, sample_labels = sampleEven(humorousData=humorousData, nonHumorousData=nonHumorousData, sample_size_per_class=(nTests/2), rand_seed=rand_seed2)

# shuffling, not necessary but helpful for watching in-progress results
rand_order = [i for i in range(len(sample_inputs))]
random.seed(rand_seed3)
random.shuffle(rand_order)

inputs = [sample_inputs[i] for i in rand_order]
labels = [sample_labels[i] for i in rand_order]

Sample input

In [13]:
inputs[-1]

[{'role': 'system',
  'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."},
 {'role': 'user',
  'content': 'Use the following as context:\n\nThe Scene is: Outside Penny’s door.\nPenny says "Do you know where I was all morning? Auditioning with 50 other blondes for some stupid antidepressant commercial. And for what? So I\'ll finally get my daddy\'s approval?" to Leonard.\nLeonard says "- Did you get the part? " to Penny.\nLeonard says "You want to talk about not getting love from a parent. You know what I used to do when I was little to have some sensation of human contact?" to Penny.\nPenny says "Yeah, you grabbed your penis and wouldn\'t let go." to Leonard.\nPenny says "Your mother told me." to Leonard.\n\n\nBased on the context provided, is the following line from the sitc

# The LLAMA 3 8B instruct model

In [ ]:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(model_id, token=userdata.get('HF_TOKEN'))
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     token=userdata.get('HF_TOKEN')
# )

In [11]:
!pip install bitsandbytes
from transformers import BitsAndBytesConfig, pipeline

# Loads Model from Drive

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          token=userdata.get('HF_TOKEN'),
                                          low_cpu_mem_usage=True,
                                          quantization_config=quantization_config,
                                          )

model_path = '/content/drive/My Drive/INFO4940_humor/LLAMA3_8b_instruct_Quantized_4bit'
model = AutoModelForCausalLM.from_pretrained(model_path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.7 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Testing

In [14]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


input = inputs[0]
label = labels[0]

input_ids = tokenizer.apply_chat_template(
  input,
  add_generation_prompt=True,
  return_tensors="pt"
).to(model.device)

outputs = model.generate(
  input_ids,
  max_new_tokens=50,
  eos_token_id=terminators,
  do_sample=True,
  temperature=0.3,
  top_p=0.9,
  num_return_sequences=1,
)

response = outputs[0][input_ids.shape[-1]:]
response = tokenizer.decode(response, skip_special_tokens=True)

print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


1


In [15]:
print(input)
print(f'expected: {label}, model: {response}')

[{'role': 'system', 'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."}, {'role': 'user', 'content': 'Use the following as context:\n\nThe Scene is: The cafeteria.\nSheldon says "Professor Jones told the symposium he had a new method for calculating the mass of a muon. Five times the limit of (laughs) \'E\' to the upsilon as..." to Howard, Leonard and Raj.\nSheldon says "Okay. No, no. I\'ll start over. Professor..." to Howard, Leonard and Raj.\nHoward says "I haven\'t seen him laugh that hard since the day Leonard made that multiplication error." to Sheldon, Leonard and Raj.\nSheldon says "Oh, Oh, Lord, that multiplication error! He thought he carried the one. But he didn\'t." to Howard, Leonard and Raj.\nLeonard says "It\'s not funny. That mistake got published." to Sheldon,

In [16]:
from collections import Counter
Counter(labels)

Counter({1: 50, 0: 50})

In [17]:
# import warnings
# warnings.filterwarnings("ignore", message="The attention mask and the pad token id were not set.*")

In [18]:
# nTests = 100
print("NCONTEXT: ", nContext)
print("NTESTS", nTests)

results, resultsData, pred, report = testLLAMA(inputs, labels, printResultsEveryN=10)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


NCONTEXT:  5
NTESTS 100


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Test: 1 of  100
Success rate: 100.00%

current results: {'truePos': 1, 'trueNeg': 0, 'falsePos': 0, 'falseNeg': 0, 'other': 0}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

...

Test: 81 of  100
Success rate: 59.26%

current results: {'truePos': 41, 'trueNeg': 7, 'falsePos': 30, 'falseNeg': 3, 'other': 0}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

Test: 91 of  100
Success rate: 52.75%

current results: {'truePos': 41, 'trueNeg': 7, 'falsePos': 40, 'falseNeg': 3, 'other': 0}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end gene

Final success rate: 52.00%
final results: {'truePos': 45, 'trueNeg': 7, 'falsePos': 43, 'falseNeg': 5, 'other': 0}


In [19]:

true = [str(i) for i in labels]
print("N CONTEXT:", nContext)
report = classification_report(y_pred=pred, y_true=true, target_names=["Non-Humorous","Humorous"])

print(report)

N CONTEXT: 5
              precision    recall  f1-score   support

Non-Humorous       0.58      0.14      0.23        50
    Humorous       0.51      0.90      0.65        50

    accuracy                           0.52       100
   macro avg       0.55      0.52      0.44       100
weighted avg       0.55      0.52      0.44       100



# Some interesting outputs along the way:

## user: "**What do you know about the sitcom the Big Bang Theory?**"

A popular and iconic sitcom! The Big Bang Theory is a American sitcom that originally aired from 2007 to 2019. Created by Chuck Lorre and Bill Prady, the show is a comedy about a group of socially awkward and intellectually brilliant scientists and their neighbor, a waitress and aspiring actress.

The show revolves around the lives of five main characters:

1. Sheldon Cooper (played by Jim Parsons), a theoretical physicist with Asperger's syndrome and a strict adherence to routine.
2. Leonard Hofstadter (played by Johnny Galecki), an experimental physicist and Sheldon's roommate and best friend.
3. Howard Wolowitz (played by Simon Helberg), an aerospace engineer and fellow physicist who lives across the hall from Leonard and Sheldon.
4. Rajesh "Raj" Koothrappali (played by Kunal Nayyar), an astrophysicist who is unable to speak in the presence of women unless he drinks alcohol.
5. Penny (played by Kaley Cuoco), a waitress and aspiring actress who lives across the hall from Leonard and Sheldon.

Throughout the show's 12 seasons, the characters navigate their personal and professional lives, often finding themselves in humorous and absurd situations. The show explores themes such as friendship, love, and the [cut off]

-------------------

## Prompt:
system: "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."

user: 'Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:

The Scene is still: The Cafeteria
Sheldon says "It's not what it looks like." to Leonard.'

assistant: 'Can you provide some context to help me with this decision?'

user: 'Yes, use the following as context:

The Scene is still: The Cafeteria
Priya says "My brother-- he's got a big crush on Bernadette." to Leonard.
Leonard says "You're moving back to India?" to Priya.
Leonard says "What's going on?" to Raj, Penny, Raj and Howard.
Penny says "It's-It's not what it looks like." to Raj, Leonard, Raj and Howard.Sheldon says "What does it look like?" to Raj, Leonard, Raj and Howard.


## Model's Response:

Based on the context, I would classify the target moment as humorous. The phrase "It's not what it looks like" is a common idiomatic expression, but in this scene, it's being used in a way that's reminiscent of a soap opera or a melodrama, which adds a layer of humor to the situation. The fact that Sheldon is asking "What does it look like?" also implies that he's being obtuse and unaware of the obvious, which is a characteristic that's often played for comedic effect in the show.
